In [1]:
from avalanche.benchmarks import SplitMNIST

benchmark = SplitMNIST(n_experiences=2)

train_stream = benchmark.train_stream
test_stream = benchmark.test_stream

for experience in train_stream: 
    print("Start of task ", experience.task_label)
    print('Classes in this task:', experience.classes_in_this_experience)

    current_training_set = experience.dataset
    print('Task {}'.format(experience.task_label))
    print('This task contains', len(current_training_set), 'training examples')

    current_test_set = test_stream[experience.current_experience].dataset
    print('This task contains', len(current_test_set), 'test examples')


/Users/daniyarzakarin/miniconda3/envs/clenv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/daniyarzakarin/miniconda3/envs/clenv/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <5AA8DD3D-A2CC-31CA-8060-88B4E9C18B09> /Users/daniyarzakarin/miniconda3/envs/clenv/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <EEB3232B-F6A7-3262-948C-BB2F54905803> /Users/daniyarzakarin/miniconda3/envs/clenv/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/daniyarzakarin/miniconda3/envs/clenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress 

Start of task  0
Classes in this task: [0, 1, 2, 8, 9]
Task 0
This task contains 30423 training examples
This task contains 5130 test examples
Start of task  0
Classes in this task: [3, 4, 5, 6, 7]
Task 0
This task contains 29577 training examples
This task contains 4870 test examples


Defining the file logger

In [11]:
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import forgetting_metrics, \
    accuracy_metrics, loss_metrics, timing_metrics, cpu_usage_metrics, \
    confusion_matrix_metrics, disk_usage_metrics, StreamConfusionMatrix
from avalanche.logging import InteractiveLogger

text_logger = InteractiveLogger()
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    StreamConfusionMatrix(num_classes=benchmark.n_classes, save_image=False),
    loggers=[text_logger]
)

Models and Strategies

In [12]:
from avalanche.models import SimpleMLP
from avalanche.training import Naive, EWC
from torch.optim import SGD
from torch.nn import CrossEntropyLoss

mlp_naive = SimpleMLP(num_classes=benchmark.n_classes)
naive_strategy = Naive(
    model = mlp_naive, 
    optimizer = SGD(mlp_naive.parameters(), lr=0.001, momentum=0.9),
    criterion = CrossEntropyLoss(), 
    train_mb_size=500, 
    train_epochs=10, 
    eval_mb_size=100,
    evaluator=eval_plugin)

mlp_ewc = SimpleMLP(num_classes=benchmark.n_classes)
ewc_strategy = EWC(
    model = mlp_ewc, 
    optimizer = SGD(mlp_ewc.parameters(), lr=0.001, momentum=0.9),
    criterion = CrossEntropyLoss(), 
    ewc_lambda = 2000,
    train_mb_size=500, 
    train_epochs=10, 
    eval_mb_size=100,
    evaluator=eval_plugin)



Training

In [13]:
results_naive = []
print('Starting experiment...')

for exp_id, experience in enumerate(train_stream):
    print("Start of experience ", experience.current_experience)

    naive_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the current test set')
    results_naive.append(naive_strategy.eval(benchmark.test_stream[exp_id]))

print('Final evaluation...')
naive_strategy.eval(benchmark.test_stream)

Starting experiment...
Start of experience  0
-- >> Start of training phase << --
100%|██████████| 61/61 [00:04<00:00, 13.02it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.1672
	Loss_MB/train_phase/train_stream/Task000 = 0.5613
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6955
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.8558
100%|██████████| 61/61 [00:05<00:00, 11.92it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4146
	Loss_MB/train_phase/train_stream/Task000 = 0.3255
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8949
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9149
100%|██████████| 61/61 [00:05<00:00, 11.68it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.3070
	Loss_MB/train_phase/train_stream/Task000 = 0.2863
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9110
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9267
100%|██████████| 61/61 [00:04<00:00, 13.08it/s]
Epoch 3 ended.
	Loss_Epoch/

{'Top1_Acc_MB/train_phase/train_stream/Task000': 0.974025974025974,
 'Loss_MB/train_phase/train_stream/Task000': 0.10743999481201172,
 'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9396828616830646,
 'Loss_Epoch/train_phase/train_stream/Task000': 0.19612806596527696,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.014230019493177388,
 'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 5.965535226156372,
 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.4699,
 'Loss_Stream/eval_phase/test_stream/Task000': 3.138834893092513,
 'StreamForgetting/eval_phase/test_stream': 0.9405458089668616,
 'ConfusionMatrix_Stream/eval_phase/test_stream': tensor([[  0,   0,   0,  38,   0, 860,  73,   9,   0,   0],
         [  0,  73,   0, 671,  10, 143,  17, 221,   0,   0],
         [  0,   0,   0, 514,  64,  34, 370,  50,   0,   0],
         [  0,   0,   0, 959,   2,  21,   4,  24,   0,   0],
         [  0,   0,   0,   1, 956,   1,  17,   7,   0,   0],
         [  0,   0,   0,  49,  14, 

In [14]:
results_ewc = []
print('Starting experiment...')

for exp_id, experience in enumerate(train_stream):
    print("Start of experience ", experience.current_experience)

    ewc_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the current test set')
    results_ewc.append(ewc_strategy.eval(benchmark.test_stream[exp_id]))

print('Final evaluation...')
ewc_strategy.eval(benchmark.test_stream)

Starting experiment...
Start of experience  0
-- >> Start of training phase << --
100%|██████████| 61/61 [00:04<00:00, 13.46it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.2564
	Loss_MB/train_phase/train_stream/Task000 = 0.5641
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6558
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.8842
100%|██████████| 61/61 [00:04<00:00, 13.94it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4232
	Loss_MB/train_phase/train_stream/Task000 = 0.3516
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8949
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9243
100%|██████████| 61/61 [00:04<00:00, 13.79it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.3088
	Loss_MB/train_phase/train_stream/Task000 = 0.2859
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9129
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9125
100%|██████████| 61/61 [00:04<00:00, 13.86it/s]
Epoch 3 ended.
	Loss_Epoch/

{'Top1_Acc_MB/train_phase/train_stream/Task000': 0.922077922077922,
 'Loss_MB/train_phase/train_stream/Task000': 0.21081849932670593,
 'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9380599790377658,
 'Loss_Epoch/train_phase/train_stream/Task000': 0.23651596240989736,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.10818713450292397,
 'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 3.9334067754578173,
 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.5186,
 'Loss_Stream/eval_phase/test_stream/Task000': 2.0981782685741783,
 'StreamForgetting/eval_phase/test_stream': 0.849317738791423,
 'ConfusionMatrix_Stream/eval_phase/test_stream': tensor([[  0,   0,   0,  40,   1, 836,  96,   7,   0,   0],
         [  0, 497,   0, 468,   6,  37,  11, 116,   0,   0],
         [  0,   0,  58, 523,  65,  32, 304,  50,   0,   0],
         [  0,   0,   0, 955,   2,  23,   4,  26,   0,   0],
         [  0,   0,   0,   1, 955,   3,  16,   7,   0,   0],
         [  0,   0,   0,  43,  16, 